<section style="direction: ltr; text-align: center; margin: 30px;">

# **SQL-Fundamentals-Farsi**

</section>

<section style="direction: ltr; text-align: justyfy; margin: 30px;">

This project offers a versatile and automated solution for generating database reports in structured formats. By leveraging popular Python libraries such as Pandas, SQLAlchemy, and Openpyxl, it simplifies the process of extracting, displaying, and exporting data from SQL Server databases. With the added ability to handle both Excel and CSV outputs, it provides a flexible tool for data analysis and reporting needs.

</section>

---


### **Calling the SQL Server Connection Function**


In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


def create_connection(server, database, username=None, password=None):
    """
    Create a connection to the SQL Server database using SQLAlchemy.

    Parameters:
    - server: str, the name of the SQL Server
    - database: str, the name of the database
    - username: str, optional, SQL Server username (for SQL authentication)
    - password: str, optional, SQL Server password (for SQL authentication)

    Returns:
    - engine: sqlalchemy.engine.base.Engine, the SQLAlchemy engine object or None if the connection fails
    """
    try:
        if username and password:
            connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
        else:
            connection_string = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

        engine = create_engine(connection_string)

        # Test connection
        with engine.connect() as conn:
            logging.info("Connection successful!")

        return engine

    except SQLAlchemyError as e:
        logging.error(f"Connection failed: {e}")
        return None

---

### **Check if the database connection is successful**


In [2]:
def check_connection(engine):
    if engine:
        logging.info("Database connection established successfully.")
        logging.info(f"Server: {server}")
        logging.info(f"Database: {database}")
        return True
    else:
        logging.error("Failed to establish database connection.")
        return False


# Call the create_connection function
server = r"imsalione-pc\imsalionedb"
database = "EnergyCompany"

engine = create_connection(server, database)

# Use the check_connection function to verify the connection
if check_connection(engine):
    # Proceed with database operations
    pass
else:
    # Handle the failed connection
    pass

2024-11-04 23:37:46,497 - INFO - Connection successful!
2024-11-04 23:37:46,498 - INFO - Database connection established successfully.
2024-11-04 23:37:46,498 - INFO - Server: imsalione-pc\imsalionedb
2024-11-04 23:37:46,500 - INFO - Database: EnergyCompany


---

### **Fetching and Displaying Data from SQL Server**


In [14]:
import pandas as pd
from sqlalchemy import exc
from pathlib import Path
from prettytable import PrettyTable

# Define the directory where you want to save the report files
output_directory = Path("C:/Reports")  # Adjust the path as needed
output_directory.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists


def truncate(value, width):
    return str(value)[:width] + "..." if len(str(value)) > width else str(value)


def fetch_and_display_queries(
    engine,
    queries,
    pnum=10,
    export_to_excel=False,
    export_to_csv=False,
    display_tail=False,
):
    """
    Fetch data from multiple SQL queries and optionally export to Excel or CSV.

    Parameters:
    - engine: SQLAlchemy engine object.
    - queries: List of SQL query strings.
    - pnum: int, number of rows to display (default is 10).
    - export_to_excel: bool, whether to export data to an Excel file (default is False).
    - export_to_csv: bool, whether to export data to a CSV file (default is False).
    - display_tail: bool, whether to display the last pnum rows (default is False).
    """

    if engine:
        # If exporting to Excel, create the writer object for the single file
        if export_to_excel:
            file_path = (
                output_directory
                / f"report_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}.xlsx"
            )
            writer = pd.ExcelWriter(file_path, engine="openpyxl")

        for i, query in enumerate(queries, 1):

            print(f"Executing query {i}:\n{query}")
            try:
                # Load data into a pandas DataFrame
                df = pd.read_sql(query, engine)

                # Display the first or last pnum rows based on display_tail parameter
                table = PrettyTable()
                table.field_names = df.columns

                if display_tail:
                    rows = df.iloc[-pnum:].itertuples(index=False)
                else:
                    rows = df.iloc[:pnum].itertuples(index=False)

                for row in rows:
                    table.add_row([truncate(value, 25) for value in row])

                print(table)

                # Print the number of records in the query result
                record_count = len(df)
                print(f"Query {i} returned {record_count} records.\n")

                # Export to Excel in different sheets if requested
                if export_to_excel:
                    sheet_name = f"Query_{i}"
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(
                        f"Data from query {i} exported to sheet {sheet_name} in {file_path}"
                    )

                # Export to CSV if requested (each query in a different CSV file)
                if export_to_csv:
                    csv_file_path = (
                        output_directory
                        / f"report_{i}_{pd.Timestamp.now().strftime('%d%m%Y%H%M%S')}.csv"
                    )
                    df.to_csv(csv_file_path, index=False)
                    print(f"Data from query {i} exported to CSV at {csv_file_path}")

            except exc.SQLAlchemyError as db_error:
                print(f"Database error occurred during query {i}: {db_error}")
            except Exception as e:
                print(f"An error occurred during query {i}: {e}")

        # Save the Excel file with all sheets after the loop finishes
        if export_to_excel:
            writer.close()  # Use close() instead of save()
            print(f"All data exported to {file_path}")

    else:
        print("No database connection.")


# Example usage:
# queries = ["""SELECT COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'Customer';""", ...]
# fetch_and_display_multiple_queries(engine, queries, pnum=20, export_to_excel=True, export_to_csv=True, sidplay_tail=True)

---

### **1- Executing SQL Query to Fetch and Display Table Information**


In [15]:
queries = [
    """SELECT DISTINCT [معاونت] FROM table1"""
]

fetch_and_display_queries(engine, queries, pnum=20)

Executing query 1:
SELECT DISTINCT [معاونت] FROM table1
+------------------------------+
|            معاونت            |
+------------------------------+
|            تولید             |
|     ديسپاچينگ و مخابرات      |
|           زيرساخت            |
|           زیرساخت            |
| مركز مطالعات سیستم و انرژ... |
|          نفت و گاز           |
+------------------------------+
Query 1 returned 6 records.



---

### **2- Executing SQL Query to Fetch and Display Table Information**


In [21]:
query = ["""
         SELECT COUNT(*) AS NumberOfProjects, [معاونت]
         FROM table1
         GROUP BY [معاونت]
         ORDER BY NumberOfProjects DESC
         """]

fetch_and_display_queries(engine, query)

Executing query 1:

         SELECT COUNT(*) AS NumberOfProjects, [معاونت]
         FROM table1
         GROUP BY [معاونت]
         ORDER BY NumberOfProjects DESC
         
+------------------+------------------------------+
| NumberOfProjects |            معاونت            |
+------------------+------------------------------+
|       108        |           زیرساخت            |
|        31        |            تولید             |
|        31        |     ديسپاچينگ و مخابرات      |
|        28        |          نفت و گاز           |
|        6         | مركز مطالعات سیستم و انرژ... |
|        5         |           زيرساخت            |
+------------------+------------------------------+
Query 1 returned 6 records.

